# 0 IMPORTS

## 0.2 Loading Data

In [4]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

df_sales_raw = pd.read_csv( 'data/train.csv', low_memory=False )
df_store_raw = pd.read_csv( 'data/store.csv', low_memory=False )

# merge
df_raw = pd.merge( df_sales_raw, df_store_raw, how='left', on='Store')

# 10 DEPLOY MODEL TO PRODUCTION

## 10.3 API Tester

In [5]:
import requests
import json

In [6]:
# loading test dataset
df10 = pd.read_csv( 'C:/Users/victo/comunidade_ds/repos/Rossmann-Stores-Sales/data/test.csv' )

# merge test + store dataset
df_test = pd.merge( df10, df_store_raw, how='left', on='Store' )

# choose store for prediction
df_test = df_test[ df_test['Store'].isin( [ 12, 22 ] ) ]

# basic cleaning
df_test = df_test[ df_test['Open'] != 0 ]
df_test = df_test[ ~df_test['Open'].isnull() ] # '~df_test['Open'].isnull()' means 'taking anything other than df_test['Open'].isnull()'
df_test = df_test.drop( 'Id', axis=1 )

# convert Dataframe to json
data = json.dumps( df_test.to_dict( orient='records' ) )

In [8]:
# API Call
# local
url = 'http://10.0.0.23:5000/rossmann/predict'

# web
#url = '.../rossmann/predict'

header = {'Content-type': 'application/json'}
data = data

r = requests.post( url, data=data, headers=header )
print( 'Status Code {}'.format( r.status_code ) )

Status Code 200


In [9]:
d1 = pd.DataFrame( r.json(), columns=r.json()[0].keys() )

d2 = d1[[ 'store', 'prediction' ]].groupby('store').sum().reset_index()

for i in range( len( d2 ) ):
    print( 'Store Number {} will sell ${:,.2f} in the next 6 weeks.'.format( d2.loc[ i, 'store' ],
                                                                       d2.loc[ i, 'prediction' ] ) )

Store Number 12 will sell $242,805.44 in the next 6 weeks.
Store Number 22 will sell $184,507.55 in the next 6 weeks.
